# Radial Velocity Procedure, Goodman x Goodman

In [1]:
#Load packages and basic functions.

#from YepFunctions import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 18,'lines.linewidth':4})
import matplotlib.cm as cm
from os.path import exists
from astropy.io import fits
from astropy.time import Time
from scipy.signal import savgol_filter

from lmfit import Model #for Azmain's code
from PyAstronomy import pyasl
from scipy.signal import argrelextrema
import time
import glob
from getpass import getuser

print('Working on '+getuser()+'.')

# #directories:
# ddir='/Users/'+getuser()+'/gsu-thesis/Research/PythonPlots/data2/' #where to access and save your data
# pdir='/Users/'+getuser()+'/gsu-thesis/Research/PythonPlots/plots2/' #where to save your plots
cdir='/Users/'+getuser()+'/gsu-thesis/Research/chiron/' #reduced chiron data, straight from GSU server
# ndir='/Users/Cougy/gsu-thesis/Research/chiron/normalized/' #reduced chiron data, straight from GSU server

#Ultimate opendat:
def opendatt(dir,filename,spl=''): #dir,'filename'. For opening a data file. Can then send through roundtable.
    f=open(dir+filename,'r')
    dat=f.readlines()
    f.close()
    if spl=='':
        labels=dat[0][0:-1].split()
        dat2=[[a.strip('\n') for a in d.split()] for d in dat if d[0]!='#']
    else:
        labels=dat[0][0:-1].split(spl)
        dat2=[[a.strip('\n') for a in d.split(spl)] for d in dat if d[0]!='#']
    dat3=[['nan' if a.strip()=='' else a for a in d] for d in dat2]
    return [dat3,labels]

def opendat(dirr,filename,params,splitchar='',prin='y'): #Use as var,var,var...=opendat(dir,'filename',['keys']).
    if splitchar=='':
        dat,label=opendatt(dirr,filename)
    else:
        dat,label=opendatt(dirr,filename,splitchar)  #Get keys by first leaving ['keys'] blank: opendat(dirr,filename,[])
    if prin=='y':
        print(label)
    varrs=[]
    for i in range(len(params)):
        j=label.index(params[i])
        try:
            var=np.array([float(d[j]) for d in dat]) #works for float.
            varrs.append(var)
        except ValueError:
            var=[d[j].strip() for d in dat] #works for strings.
            varrs.append(var)
    if len(params)==1:
        varrs=varrs[0]
    return varrs

def writedat(dirr,filename,pars,label): #.dat auto included. pars as [name,ra,dec] etc.
    datp=[[str(a[i]) for a in pars] for i in range(len(pars[0]))]
    f=open(dirr+filename+'.dat','w')
    print('\t'.join(label),file=f)
    print(label)
    for d in datp:
        print('\t'.join(d),file=f)
    f.close()
    print('It is written: '+filename+'.dat')

def hmsdms(ra,dec,splitchar=' '): #input in degrees, output in HH MM S.SSS
    H=ra/15.
    h=int(H)
    M=(H-h)*60.
    m=int(M)
    s=(M-m)*60.
    
    DEC=abs(dec)
    sign=np.sign(dec)
    dd=int(DEC)
    d=int(sign*dd)
    AM=(DEC-dd)*60.
    am=int(AM)
    ass=(AM-am)*60.
    return str(h)+splitchar+str(m)+splitchar+str(round(s,3))+'\t'+str(d)+splitchar+str(am)+splitchar+str(round(ass,3))

def hmsdms2(ra,dec,splitchar=' '):
    RADEC=[hmsdms(ra[i],dec[i],splitchar) for i in range(len(ra))]
    return RADEC

def dmshms(ra,dec): #input HH:MM:S.SSS, output degrees
    if ':' in ra[0]:
        splitchar=':'
    else:
        splitchar=' '
    hms=[a.split(splitchar) for a in ra]
    h=[float(a[0])*15. for a in hms]
    m=[float(a[1])*15./60. for a in hms]
    s=[float(a[2])*15./3600. for a in hms]
    H=[h[i]+m[i]+s[i] for i in range(len(ra))]
    
    dms=[a.split(splitchar) for a in dec]
    d=[float(a[0]) for a in dms]
    am=[float(a[1])/60. for a in dms]
    ass=[float(a[2])/3600. for a in dms]
    sign=[-1. if a[0]=='-' else 1. for a in dec]
    D=[sign[i]*(abs(d[i])+am[i]+ass[i]) for i in range(len(ra))]
    return H,D

def dmshms1(ra,dec): #input HH:MM:S.SSS, output degrees
    if ':' in ra:
        splitchar=':'
    else:
        splitchar=' '
    hms=ra.split(splitchar)
    h=float(hms[0])*15.
    m=float(hms[1])*15./60.
    s=float(hms[2])*15./3600.
    H=h+m+s
    
    dms=dec.split(splitchar)
    d=float(dms[0])
    am=float(dms[1])/60.
    ass=float(dms[2])/3600.
    if dec[0]=='-':
        sign=-1.
    else:
        sign=1.
    D=sign*(abs(d)+am+ass)
    return H,D

def erm(val,err): #list,list
    v=np.array(val)
    e=np.array(err)
    w=1.0/e**2.0
    avg=np.nansum(w*v)/np.nansum(w)
    avgerr=1.0/np.sqrt(np.nansum(w))
    return avg,avgerr

Working on Cougy.


In [2]:
#Functions

#crosscorrRV routine with normalization:

import numpy as np
from PyAstronomy.pyaC import pyaErrors as PE
from PyAstronomy.pyasl import _ic
from lmfit import Model #for Azmain's code

def crosscorrRVn(w, f, tw, tf, rvmin, rvmax, drv, mode="normdop", skipedge=0, edgeTapering=None):
    """
        Cross-correlate a spectrum with a template.
        
        The algorithm implemented here works as follows: For
        each RV shift to be considered, the wavelength axis
        of the template is shifted, either linearly or using
        a proper Doppler shift depending on the `mode`. The
        shifted template is then linearly interpolated at
        the wavelength points of the observation
        (spectrum) to calculate the cross-correlation function.
        
        Parameters
        ----------
        w : array
                The wavelength axis of the observation.
        f : array
                The flux axis of the observation.
        tw : array
                The wavelength axis of the template.
        tf : array
                The flux axis of the template.
        rvmin : float
                Minimum radial velocity for which to calculate
                the cross-correlation function [km/s].
        rvmax : float
                Maximum radial velocity for which to calculate
                the cross-correlation function [km/s].
        drv : float
                The width of the radial-velocity steps to be applied
                in the calculation of the cross-correlation
                function [km/s].
        mode : string, {lin, doppler}, optional
                The mode determines how the wavelength axis will be
                modified to represent a RV shift. If "lin" is specified,
                a mean wavelength shift will be calculated based on the
                mean wavelength of the observation. The wavelength axis
                will then be shifted by that amount. If "doppler" is
                specified (the default), the wavelength axis will
                properly be Doppler-shifted.
        skipedge : int, optional
                If larger zero, the specified number of bins will be
                skipped from the begin and end of the observation. This
                may be useful if the template does not provide sufficient
                coverage of the observation.
        edgeTapering : float or tuple of two floats
                If not None, the method will "taper off" the edges of the
                observed spectrum by multiplying with a sine function. If a float number
                is specified, this will define the width (in wavelength units)
                to be used for tapering on both sides. If different tapering
                widths shall be used, a tuple with two (positive) numbers
                must be given, specifying the width to be used on the low- and
                high wavelength end. If a nonzero 'skipedge' is given, it
                will be applied first. Edge tapering can help to avoid
                edge effects (see, e.g., Gullberg and Lindegren 2002, A&A 390).
        
        Returns
        -------
        dRV : array
                The RV axis of the cross-correlation function. The radial
                velocity refer to a shift of the template, i.e., positive
                values indicate that the template has been red-shifted and
                negative numbers indicate a blue-shift of the template.
                The numbers are given in km/s.
        CC : array
                The cross-correlation function.
    """
    if not _ic.check["scipy"]:
        raise(PE.PyARequiredImport("This routine needs scipy (.interpolate.interp1d).", \
                                                             where="crosscorrRV", \
                                                             solution="Install scipy"))
    import scipy.interpolate as sci
    # Copy and cut wavelength and flux arrays
    w, f = w.copy(), f.copy()
    if skipedge > 0:
        w, f = w[skipedge:-skipedge], f[skipedge:-skipedge]
    
    if edgeTapering is not None:
        # Smooth the edges using a sine
        if isinstance(edgeTapering, float):
            edgeTapering = [edgeTapering, edgeTapering]
        if len(edgeTapering) != 2:
            raise(PE.PyAValError("'edgeTapering' must be a float or a list of two floats.", \
                                                     where="crosscorrRV"))
        if edgeTapering[0] < 0.0 or edgeTapering[1] < 0.0:
            raise(PE.PyAValError("'edgeTapering' must be (a) number(s) >= 0.0.", \
                                                     where="crosscorrRV"))
        # Carry out edge tapering (left edge)
        indi = np.where(w < w[0]+edgeTapering[0])[0]
        f[indi] *= np.sin((w[indi] - w[0])/edgeTapering[0]*np.pi/2.0)
        # Carry out edge tapering (right edge)
        indi = np.where(w > (w[-1]-edgeTapering[1]))[0]
        f[indi] *= np.sin((w[indi] - w[indi[0]])/edgeTapering[1]*np.pi/2.0 + np.pi/2.0)
    
    # Speed of light in km/s
    c = 299792.458
    # Check order of rvmin and rvmax
    if rvmax <= rvmin:
        raise(PE.PyAValError("rvmin needs to be smaller than rvmax.",
                                                 where="crosscorrRV", \
                                                 solution="Change the order of the parameters."))
    # Check whether template is large enough
    if mode == "lin":
        meanWl = np.mean(w)
        dwlmax = meanWl * (rvmax/c)
        dwlmin = meanWl * (rvmin/c)
        if (tw[0] + dwlmax) > w[0]:
            raise(PE.PyAValError("The minimum wavelength is not covered by the template for all indicated RV shifts.", \
                                                     where="crosscorrRV", \
                                                     solution=["Provide a larger template", "Try to use skipedge"]))
        if (tw[-1] + dwlmin) < w[-1]:
            raise(PE.PyAValError("The maximum wavelength is not covered by the template for all indicated RV shifts.", \
                                                     where="crosscorrRV", \
                                                     solution=["Provide a larger template", "Try to use skipedge"]))
    elif mode == "doppler" or mode == 'normdop':
        # Ensure that the template covers the entire observation for all shifts
        maxwl = tw[-1] * (1.0+rvmin/c)
        minwl = tw[0] * (1.0+rvmax/c)
        if minwl > w[0]:
            raise(PE.PyAValError("The minimum wavelength is not covered by the template for all indicated RV shifts.", \
                                                     where="crosscorrRV", \
                                                     solution=["Provide a larger template", "Try to use skipedge"]))
        if maxwl < w[-1]:
            raise(PE.PyAValError("The maximum wavelength is not covered by the template for all indicated RV shifts.", \
                                                     where="crosscorrRV", \
                                                     solution=["Provide a larger template", "Try to use skipedge"]))
    else:
        raise(PE.PyAValError("Unknown mode: " + str(mode), \
                                                 where="crosscorrRV", \
                                                 solution="See documentation for available modes."))
    # Calculate the cross correlation
    drvs = np.arange(rvmin, rvmax, drv)
    cc = np.zeros(len(drvs))
    for i, rv in enumerate(drvs):
        if mode == "lin":
            # Shift the template linearly
            fi = sci.interp1d(tw+meanWl*(rv/c), tf)
            cc[i] = np.sum(f * fi(w))
        elif mode == "doppler":
            # Apply the Doppler shift
            fi = sci.interp1d(tw*(1.0 + rv/c), tf)
            cc[i] = np.sum(f * fi(w))
        # Shifted template evaluated at location of spectrum
        elif mode == "normdop":
            # Apply the Doppler shift
            fi = sci.interp1d(tw*(1.0 + rv/c), tf)
        # Shifted template evaluated at location of spectrum
            cc[i] = 1./float(len(w)-skipedge)*np.sum((f-np.mean(f)) * (fi(w)-np.mean(fi(w))))/np.sqrt(np.var(f)*np.var(fi(w)))
    return drvs, cc

def sigmavg(dat,head,w,f,fn): #fits data, fits header, wavelength, unnormalized flux, normalized flux; Goodman
    #w,f=wf(dat,o)
    gain=float(head['GAIN'])
    RN=float(head['RDNOISE'])
#     if head['MODES'].split(',')[int(head['MODE'])].strip()=='fiber':
#         K=2.5
#     elif head['MODES'].split(',')[int(head['MODE'])].strip()=='slicer':
#         K=9.
    K=5 #pixels binned across spectral order. Here, it's...? No clue.
    SNR=np.array(f)*gain/np.sqrt(np.array(f)*gain+K*RN**2.)
    c=299792.458 #km/s
    dfdw=deriv(w,fn)
    Sigmav=1./np.sqrt(np.nansum((np.array(dfdw)*np.array(w)*np.array(SNR)/c)**2.))
    return Sigmav

#By chain rule: df/dv = df/dw * w/c. So can use df/dw derivative and multiply by w/c for deriv at each pixel.
def deriv(X,Y): #numerical
    #middle: 2 derivs on either side of pixel, then average
    x0=np.array(X[:-2])
    y0=np.array(Y[:-2])
    x1=np.array(X[1:-1])
    y1=np.array(Y[1:-1])
    x2=np.array(X[2:])
    y2=np.array(Y[2:])
    dydx_m=((y1-y0)/(x1-x0)+(y2-y1)/(x1-x0))/2.
    #print(len(dydx_m))
    #ends: just do 1 deriv
    dydx_b=(Y[1]-Y[0])/(X[1]-X[0])
    dydx_e=(Y[-1]-Y[-2])/(X[-1]-X[-2])
    #combine:
    dydx=[dydx_b,]+list(dydx_m)+[dydx_e,]
    #print(len(dydx))
    return dydx

#barycentric correction
def bcvg(head): #for Goodman
    #RA,Dec,expt,jd (jd of middle of exposure time)

    # Coordinates of SOAR telescope for Goodman
    longitude = -1.*(70.+44./60.+1.11/3600.) #degrees, E=+,W=-
    latitude = -30.-14./60.-16.41/3600. #degrees
    altitude = 2713. #meters
    
    expt=head['EXPTIME'] #exposure time
    #print(expt,type(expt),type(Time(head['DATE-OBS']).jd))
    JD=Time(head['DATE-OBS']).jd+expt/2./60./24. #Julian day of middle of exposure, UT
    # Coordinates in degrees (J2000) for pyas1
    RA=[float(d)*15. for d in head['RA'].split(':')] #RA, degrees
    DEC=[float(d) for d in head['DEC'].split(':')] #Dec
    if '-' in head['DEC']:
        sign=-1.
    else:
        sign=1.
    ra2000 = RA[0]+RA[1]/60.+RA[2]/3600. #RA
    dec2000 = sign*(abs(DEC[0])+DEC[1]/60.+DEC[2]/3600.) #Dec

    # Barycentric velocity correction
    cor = pyasl.helcorr(longitude, latitude, altitude,ra2000, dec2000, JD)[0]
    return cor

def gaussian(x, amp, cen, wid): # Gaussian model
    """1-d gaussian: gaussian(x, amp, cen, wid)"""
    return ((1 * amp) / (np.sqrt(2*np.pi) * wid)) * np.exp(-(x-cen)**2 / (2*wid**2)) 
gmodel = Model(gaussian)

#new sval 2024 (allow IV), let's try only up to K7, rest go to "10" to change class. K7 goes to "8."
def sval(ss): #'spt'
    def vs(s): #v from s, for single value
        if s=='nan':
            v= np.float('nan')
            return np.float('nan')
        else:
            val=60-['O','B','A','F','G','K','M','L','T','Y'].index(s[0])*10+(10-float(s[1:].strip('V').strip('I')))
        if s[0] in ['M','L','T','Y']: #M gets set back because of K; K7 should be right next to M0
            val+=2
        if 'III' in s:
            v= val-100.
        elif 'I' in s and 'III' not in s and 'V' not in s:
            v= val-200.
        else:
            v= val
        return v
    if type(ss)==str:
        final=vs(ss)
    if type(ss)==list or type(ss)==type(np.array([1])):
        final=[]
        for i in range(len(ss)):
            s=ss[i]
            v=vs(s)
            final.append(v)
    return final

def sptfromsval(s): #'spt'
    obafgkm=['O','B','A','F','G','K','M']
    if s<=12:
        s-=2 #handle the Ms
    if s>=0:
        lum='V'
        S=int(s/10.)
    elif s<0 and s>=-100:
        lum='III'
        S=int((s+100.)/10.)
    elif s<-100:
        lum='I'
        S=int((s+200.)/10.)
    else:
        print('Error with luminosity class.')
    clas=str(round(10-(s-float(10*S)),1)).replace('.0','')
    if round(10-(s-float(10*S)),1)==10:
        clas='0'
        S-=1
    oba=obafgkm[6-S]
    return oba+clas+lum

#retrieve rv,vsini,bcv from LogCruncher standards summary file
def refv(stdname):
    name,vsini,vsinierr,rv,rverr=opendat(cdir,'standards/standards_metadata.dat',['#name','vsini', 'vsinierr', 'rv', 'rverr'],splitchar='\t',prin='n')
    #bname,barycorr=opendat2(cdir,'standards/CHIRON_standards_bestobs.dat',['name','barycorr'])
    i=name.index(stdname)
    #j=bname.index(stdname)
    return vsini[i],vsinierr[i],rv[i],rverr[i]

def refspt(stdname):
    name,spt=opendat(cdir,'standards/standards_metadata.dat',['#name','spt'],splitchar='\t',prin='n')
    i=name.index(stdname)
    return spt[i]

def vrotcombo(vstd,vbroad): #"adds" vsinis to get total vsini
    x=vstd
    y=vbroad
    r=np.sqrt(x**2.+y**2.)
    theta=np.arctan(y/x)
    v0=0.044*r #deflection from circle
    if theta<=np.pi/4.:
        vtot=r+4.*v0/np.pi*theta
    if theta>np.pi/4.:
        vtot=r-4.*v0/np.pi*theta+2.*v0
    return vtot

In [3]:
#data

#main dir:
gdir='/Users/Cougy/Desktop/_AgnesScottCollege/SummerResearch/2024Summer/Dr.-Yep-2024-summer-research/'
#Goodman normalized spectra:
gndir=gdir+'/GoodmanNormBest/'

# get all files in their various locations:
files=glob.glob(gndir+'*.fits')
#print(files)
fnamest=[a.split('/')[-1][2:].replace('.fits','').replace('dot','.') for a in files]
fnames=[a.split('/')[-1][2:].replace('.fits','').replace('_target_1','').replace('_target_2','').replace('_target_3','').replace('dot','.') for a in files]
#print('fnames',fnames) #from files

#open my metadata file
mnames,ras,decs,Vs,spts=opendat('','Goodman2020B_AYep_TargetChecklist.txt',['name', 'ra', 'dec', 'V', 'spt'],splitchar='\t')
#print('mnames',mnames)

stds=[a for a in fnames if a[0]!='C']
stdspts=[spts[mnames.index(n)] for n in stds]
ssval=[sval(a) for a in stdspts]
#print(stds)

for i in range(len(stds)):
    print(stds[i],':',stdspts[i],refspt(stds[i]))

['#check', 'number', 'name', 'ra', 'dec', 'V', 'spt']
GJ83.1 : M4.5V M4.5V
HD217357 : K7V K7V
HD42581 : M1V M1V
HD36003 : K5V K5V
HIP18280 : M0V M0V


In [4]:
# #test:
# name='GJ83.1'
# stdname='HD42581'

# pltcc='y'
# rvcent0=0
# vsinistart0=0
# skipedge0=0
# prin='y'
# plotyn='y'

def fitvsinilite(name,stdname,rvcent0=0,vsinistart0=0,skipedge0=0,prin='n',plotyn='n',pltcc='n'): #returns the essentials.
    # One target against one standard:
    #def fitvsinilite(name,stdname,rvcent0=0,vsinistart0=0,skipedge0=0,prin='n',plotyn='n',pltcc='n'): #returns the essentials.

    #For standards test, you don't account for each standard's uncert.
    #target:
    hdu=fits.open(gndir+'n_'+name+'.fits') #.fits data
    dat=hdu[0].data
    head=hdu[0].header
    hdu.close()
    bc=bcvg(head) #barycentric correction
    w,f=dat #total spectrum
    #unnorm for sigmav
    wfunfile=head['WFUNFILE']
    hdu=fits.open(gdir+wfunfile) #.fits data
    fun=hdu[0].data[1] #unnorm spectrum
    hdu.close()

    #telluric=[[6275,6315],[6470,6530],[6570,6580]]
    #dodge telluric:
    regions=[[6000,6275],[6315,6470],[6580,6800]]
    ws=[[w[i] for i in range(len(w)) if w[i]>=t[0] and w[i]<t[1]] for t in regions] #put on known good regions.
    fs=[[f[i] for i in range(len(w)) if w[i]>=t[0] and w[i]<t[1]] for t in regions] #put on known good regions.
    funs=[[fun[i] for i in range(len(w)) if w[i]>=t[0] and w[i]<t[1]] for t in regions] #put on known good regions.
    svs=[sigmavg(dat,head,ws[o],funs[o],fs[o]) for o in range(len(fs))]
    del w
    del f
    del fun

    #standard:
    hdu=fits.open(gndir+'n_'+stdname+'.fits') #.fits data
    dat0=hdu[0].data
    head0=hdu[0].header
    hdu.close()
    bc0=bcvg(head0) #barycentric correction
    w,f=dat0 #total spectrum
    #unnorm for sigmav
    wfunfile=head0['WFUNFILE']
    hdu=fits.open(gdir+wfunfile) #.fits data
    fun=hdu[0].data[1] #unnorm spectrum
    hdu.close()
    #data for standard:
    vsini0,vsini0err,rv0,rv0err=refv(stdname) #metadata
    stdspt=refspt(stdname)

    ws0=[[w[i] for i in range(len(w)) if w[i]>=t[0] and w[i]<t[1]] for t in regions] #put on known good regions.
    fs0=[[f[i] for i in range(len(w)) if w[i]>=t[0] and w[i]<t[1]] for t in regions] #put on known good regions.
    funs0=[[fun[i] for i in range(len(w)) if w[i]>=t[0] and w[i]<t[1]] for t in regions] #put on known good regions.
    svs0=[sigmavg(dat0,head0,ws0[o],funs0[o],fs0[o]) for o in range(len(fs0))]
    del w
    del f
    del fun
    
#     plt.figure()
#     plt.plot(sum(ws,[]),sum(fs,[]))
#     plt.xlabel('Wavelength (A)')
#     plt.ylabel('Normalized Flux')

    #print((w[1]-w[0])*20)
    dw=1.6 #2.8 #what is this based on??? 1.6 for CHIRON fiber, but why???? 20x wavelength res? Kinda.

    os=list(range(len(ws0)))
    #os=[0,1] #region 2 is too small
    #print('os',os)

    print('\nFor target',name,'against standard',stdname,'(rv = '+str(rv0)+' km/s, vsini = '+str(vsini0)+' km/s)')
    print('barycorr, target: ',bc,'km/s, standard:',bc0,'km/s\n')

    #default starting skipedge and edge tapering
    sk=40
    et=10

    # std+bc0 should put std at std's heliocentric rv0.
    # so, std as-is is really at rv0-bc0.
    # meanwhile, target+bc should put target at rv.
    # so, target as-is is really at rv-bc.
    # crosscorrRVn shifts the template (the std), so RVdiff is (red)shift needed to align std with target.
    # so rv0-bc0+RVdiff = rv-bc (for std as-is shifted to target as-is).
    # the velocity difference between the as-is spectra, then, is RVdiff=(rv-bc)-(rv0-bc0)=rv-rv0-bc+bc0.
    # what we want is the rv of the target: rv = RVdiff+rv0+bc-bc0
    # rv = rv0+RVdiff+bc-bc0

    # start at best guess, basically at everything-but-RVdiff: If std and targ are within 100 km/s, should be fine.
    rvcent00=rv0+rvcent0+bc-bc0

    rvs,vsinis=[],[]

    #pick vwindow for rv and vsini, based on order 1
    vwins=[]
    if pltcc=='y':
        ccs=[]
    preos=[1]
    for i in preos: #my red Goodman has 3 nonuniform "orders"
        o=i
        w,f=np.array(ws[i]),np.array(fs[i])-1. #pyasl code wants spectra normalized to 0! Weird.
        w0,f0=np.array(ws0[i]),np.array(fs0[i])-1. #""

        rv, cc = crosscorrRVn(w, f, w0, f0, rvcent00-200., rvcent00+200.,dw,skipedge=sk+int(0.5*vsinistart0)+skipedge0,edgeTapering=et+0.25*vsinistart0) #The less skipedge, the better. edgeTapering seems happy near 10.??
        maxind = np.argmax(cc)
        if pltcc=='y':
            ccs.append(cc)
        try:
            min1=np.max([a for a in argrelextrema(cc,np.less)[0] if a<maxind])
            min2=np.min([a for a in argrelextrema(cc,np.less)[0] if a>maxind])
            vwin=np.max([(min2-min1)/4.,4.]) #4 is the minimum. /4 because otherwise too broad or gaussian fits poorly
            vwins.append(vwin)
        except ValueError:
            print('order',i,'could not find peak or neighboring minima.')
            print([a for a in argrelextrema(cc,np.less)[0] if a<maxind])
            print([a for a in argrelextrema(cc,np.less)[0] if a>maxind])

    vwindow=np.nanmax([np.median(vwins),8.])
    rvcent=rv[maxind]
    vsinistart=np.max([vwindow-29.,vsinistart0])
    print('vsini window',vwindow,', vsini start',vsinistart,', rv cent',rvcent)

    if pltcc=='y':
        for i in range(len(preos)):
            plt.figure(figsize=(5,5))
            plt.title('cross-corr of order '+str(preos[i]))
            plt.plot(ccs[i])
            plt.ylabel('norm\'d cc')
            plt.xlabel('index')
            plt.plot([np.argmax(ccs[i])-int(vwindow+1.0*vsinistart),np.argmax(ccs[i])+int(vwindow+1.0*vsinistart)],[ccs[i][np.argmax(ccs[i])]*0.5,ccs[i][np.argmax(ccs[i])]*0.5])

    #find good approx. vsini and refine rvdiff
    rvdiffs,vsinibroads=[],[]
    V = np.linspace(0+vsinistart,60+vsinistart,10) # collection of vsini. Ultimately works for up to ~60+30 = 90 km/s of broadening.
    for i in preos: #[0,]:
        o=i
        w,f=np.array(ws[i]),np.array(fs[i])-1. #pyasl code wants spectra normalized to 0! Weird.
        w0,f0=np.array(ws0[i]),np.array(fs0[i])-1. #""
        rv, cc = crosscorrRVn(w, f, w0, f0, rvcent-100., rvcent+100.,dw,skipedge=sk+int(0.5*vsinistart)+skipedge0,edgeTapering=et+0.25*vsinistart) #The less skipedge, the better. edgeTapering seems happy near 10.??
        maxind = np.argmax(cc)
        #vsini
        xnew = np.linspace(w0[0],w0[-1],len(w0)) # uniform wavelength spacing
        ynew = np.interp(xnew,w0,f0) #standard spectrum interpolated to uniform wavelength spacing
        hs=[]
        for v in V: # empirical vsini relation from broadened template
            if v==0:
                v=0.01
            bflux = pyasl.rotBroad(xnew, ynew, 0.6, v) # flux of broadened spectrum
            rV, cC = crosscorrRVn(xnew, bflux, xnew, ynew,-1*int(vwindow+1.0*vsinistart),int(vwindow+1.0*vsinistart),dw,skipedge=int(sk/2+0.5*vsinistart),edgeTapering=et+0.25*vsinistart) #need to increase for faster rotation? Keep checking. CG 1's F star needs more skip and taper.
            result = gmodel.fit(cC, x=rV, amp=1., cen=0., wid=1.)
            h = result.best_values #gauss width of std with rot'l broadening of v km/s
            hs.append(h['wid'])
        width = hs #np.genfromtxt(file1) # use empirical relation to get RV and vsini
        rvmin = rv[maxind] - (vwindow+1.0*vsinistart)
        rvmax = rv[maxind] + (vwindow+1.0*vsinistart)
        rv2, cc2 = crosscorrRVn(w, f, w0, f0, rvmin, rvmax,dw,skipedge=int(sk/2+0.5*vsinistart)+skipedge0,edgeTapering=et+0.25*vsinistart) 
        maxind = np.argmax(cc2)

        if pltcc=='y':
                plt.figure(figsize=(5,5))
                plt.title('cross-corr of order '+str(o))
                plt.plot(rv2,cc2)
                plt.ylabel('norm\'d cc')
                plt.xlabel('rv')
                plt.plot([rvmin,rvmax],[cc2[np.argmax(cc2)]*0.5,cc2[np.argmax(cc2)]*0.5])

        try:
            result = gmodel.fit(cc2, x=rv2, amp=cc2[maxind], cen=rv2[maxind], wid=1.) #gauss width of actual target spectrum
            h = result.best_values
            RVdiff = h['cen'] #really the rv diff between standard and target.
            vsini = np.interp(h['wid'],width,V) #just the broadening
            #if vsini is considerably smaller than vsini_std, discard the measurement:
            #if vsini==vsini0:
            #    vsini=float('nan') #but this doesn't seem to work with my V intervals.
            #rv shouldn't change from day to day, month to month, as earth moves. So rv must be relative to point of rest.
            #bcv is motion of earth towards a star. So star's motion away from rest point is actually obs'd motion of star + calc'd motion of earth toward star.
            #u: uncorrected, c: corrected. 0:standard
            #rvc= rv0c + rvcdiff = rv0c + (rvc - rv0c) = rv0c + ((rvu + bc) - (rv0u + bc0)) = rv0c + (rvu - rv0u) + bc - bc0
            #RV_true= rv0 + RVdiff + bc - bc0
            rvdiffs.append(RVdiff)
            vsinibroads.append(vsini)
        except ValueError:
            print('Failed to fit Gaussian to order '+str(o)+'.')
            rvdiffs.append(np.float('nan'))
            vsinibroads.append(np.float('nan'))
    #Remove outliers based on median and std of middle quartile
    midrv=np.median(rvdiffs)
    midstd=np.std(np.array([d for d in rvdiffs if d>np.percentile(rvdiffs,25) and d<np.percentile(rvdiffs,75)]))
    stdfac=3.
    rvres=10. #km/s, arbitrary at the moment
    rmin=midrv-stdfac*midstd if stdfac*midstd>rvres else midrv-rvres
    rmax=midrv+stdfac*midstd if stdfac*midstd>rvres else midrv+rvres
    rvx=np.array([d for d in rvdiffs if d>rmin and d<rmax])
    bados=[os[i] for i in range(len(rvs)) if rvs[i]<=rmin or rvs[i]>=rmax]
    #weighted rv
    weights0=1./np.array(svs0)**2.
    weights0x=np.array([weights0[i] for i in range(len(rvdiffs)) if rvdiffs[i]>rmin and rvdiffs[i]<rmax])
    #normalize the weights
    weights0xn=weights0x/sum(weights0x)
    #rv
    rvxw=sum(rvx*weights0xn)
    #vsini
    vsinix=np.array([vsinibroads[i] for i in range(len(rvdiffs)) if rvdiffs[i]>rmin and rvdiffs[i]<rmax])
    #normalize the weights
    weights0xx=[weights0x[i] if np.isnan(vsinix[i])==False else float('nan') for i in range(len(weights0x))] #keep if vsini any good
    weights0xxn=weights0xx/np.nansum(weights0xx)
    vsinixw=np.nansum(vsinix*weights0xxn)

    #do full analysis with very tailored rvcent,vsinistart
    rvcent=rvxw
    vsinistart=np.max((vsinixw-10,0))
    print('new vsini start',vsinistart,', new rv cent',rvcent)
    V = np.linspace(0+vsinistart,20+vsinistart,10) # !!!!! collection of vsini. Winner: 20/10. !!!!!
    for i in range(len(os)): #[0,]:
        o=os[i]
        print(o)
        w,f=np.array(ws[i]),np.array(fs[i])-1. #pyasl code wants spectra normalized to 0! Weird.
        w0,f0=np.array(ws0[i]),np.array(fs0[i])-1. #""
        rv, cc = crosscorrRVn(w, f, w0, f0, rvcent-10.-int(vsinistart/2.), rvcent+10.+int(vsinistart/2.),dw,skipedge=sk+int(0.5*vsinistart)+skipedge0,edgeTapering=et+0.25*vsinistart) #The less skipedge, the better. edgeTapering seems happy near 10.??
        maxind = np.argmax(cc)
        #vsini
        xnew = np.linspace(w0[0],w0[-1],len(w0)) # !!!!! uniform wavelength spacing. Winner: original resolution. !!!!!
        ynew = np.interp(xnew,w0,f0)
        hs=[]
        for v in V: # empirical vsini relation from broadened template
            if v==0:
                v=0.01
            bflux = pyasl.rotBroad(xnew, ynew, 0.6, v) # flux of broadened spectrum 
            rV, cC = crosscorrRVn(xnew, bflux, xnew, ynew,-1*int(vwindow+1.0*vsinistart),int(vwindow+1.0*vsinistart),dw,skipedge=int(sk/2+0.5*vsinistart),edgeTapering=et+0.25*vsinistart) #need to increase for faster rotation? Keep checking. CG 1's F star needs more skip and taper.
            result = gmodel.fit(cC, x=rV, amp=1., cen=0., wid=1.)
            h = result.best_values
            hs.append(h['wid'])
        ###width = hs #np.genfromtxt(file1) # use empirical relation to get RV and vsini
        width=[hs[i] for i in range(len(hs)-1) if hs[i+1]-hs[i]>0.01]+[hs[-1],] #delete vertical front if present
        V2=[V[i] for i in range(len(hs)-1) if hs[i+1]-hs[i]>0.01]+[V[-1],] #match width
        rvmin = rv[maxind] - (vwindow+1.0*vsinistart)
        rvmax = rv[maxind] + (vwindow+1.0*vsinistart)
        rv2, cc2 = crosscorrRVn(w, f, w0, f0, rvmin, rvmax,dw,skipedge=int(sk/2+0.5*vsinistart)+skipedge0,edgeTapering=et+0.25*vsinistart) 
        maxind = np.argmax(cc2)
        try:
            result = gmodel.fit(cc2, x=rv2, amp=cc2[maxind], cen=rv2[maxind], wid=1.) 
            h = result.best_values
            RVdiff = h['cen'] #really the rv diff between standard and target, target-standard.
            RV_true= rv0 + RVdiff + bc - bc0
            if prin=='y':
                prince='rv diff '+str(round(RVdiff,3))+', rv = '+str(round(RV_true,5))+' km/s'
            rvs.append(RV_true)
            ###vsini = np.interp(h['wid'],width,V+vsini0) #map to vsini unbroadened = vsini_std
            ###new vsini###
            if h['wid']>np.min(width) and h['wid']<np.max(width):
                vsinibroad = np.interp(h['wid'],width,V2) #map to vsini unbroadened = vsini_std. #based on the relation of gauss widths to v broadenings + vsini_std generated earlier, the actual width of the target should correspond to THIS vsini.
                vsini=vrotcombo(vsini0,vsinibroad)
                fitpass='y'
                if prin=='y':
                    print(str(o)+': '+prince,'\tvsini =',round(vsini,3),'km/s')
                x,y,x_new,y_new,xnew,ynew=float('nan'),float('nan'),float('nan'),float('nan'),float('nan'),float('nan')
            #new vsini from fitted func, only if a little off the chart
            elif h['wid']>np.min(width)-2. and h['wid']<np.max(width)+2.:
                X,Y=width,V2 #+vsini0
                points=np.array([(X[i],Y[i]) for i in range(len(X))])
                # get x and y vectors
                x = points[:,0]
                y = points[:,1]
                # calculate polynomial
                z = np.polyfit(x, y, 2)
                f = np.poly1d(z)
                # calculate new x's and y's
                x_new = np.linspace(np.min(width)-2, np.max(width)+2, len(width))
                y_new = f(x_new)
                xnew=h['wid']
                ynew=f(xnew)
                ###
                vsinibroad=ynew
                vsini=vrotcombo(vsini0,vsinibroad)
                fitpass='y'
                if prin=='y':
                    print(str(o)+': '+prince,'\tvsini =',round(vsini,3),'km/s')
            else:
                vsinibroad=float('nan')
                vsini=float('nan')
                if prin=='y':
                    print(str(o)+': '+prince,'\tvsini =',round(vsini,3), "km/s - Skipping order",o,'for vsini.')
                x,y,x_new,y_new,xnew,ynew=float('nan'),float('nan'),float('nan'),float('nan'),float('nan'),float('nan')
                fitpass='n'
            vsinis.append(vsini)

        except ValueError:
            print('Failed to fit Gaussian to order '+str(o)+'.')
            rvs.append(np.float('nan'))
            vsinis.append(np.float('nan'))

        if plotyn=='y': #o==24 and 
            plt.figure(figsize=(5,5))
            plt.plot(width,V2,marker='o')
            plt.scatter(h['wid'],vsinibroad,color='red')
            plt.ylabel('vsini broadening')
            plt.xlabel('width')
            plt.title('From '+stdname+', vsini_std = '+str(round(vsini0,2))+' km/s')
            plt.plot(x,y,'o', x_new, y_new,color='green',ls='--',lw=1.5)
            plt.scatter(xnew,ynew,color='red')
            if fitpass=='n':
                plt.scatter(h['wid'],vsini,marker='x',lw=1.2,color='black',s=90)

    #Remove outliers based on median and std of middle quartile
    rvx=rvs #only 2 for Goodman, don't drop any "orders"
    #weighted rv
    weights0x=1./np.array(svs0)**2.
    #weights0x=np.array([weights0[i] for i in range(len(rvs)) if rvs[i]>rmin and rvs[i]<rmax])
    #normalize the weights
    weights0xn=weights0x/sum(weights0x)
    #rv
    rvxw=np.nansum(rvx*weights0xn)
    rvxstd=np.nanstd(rvx) # !!!!! standard

    #vsini
    vsinix=vsinis #np.array([vsinis[i] for i in range(len(rvs)) if rvs[i]>rmin and rvs[i]<rmax])
    #normalize the weights
    weights0xx=[weights0x[i] if np.isnan(vsinix[i])==False else float('nan') for i in range(len(weights0x))] #keep if vsini any good
    weights0xxn=weights0xx/np.nansum(weights0xx)
    vsinixw=np.nansum(vsinix*weights0xxn)
    if np.isnan(vsini0err)==True:
        vsini0err=1. #arbitrary
        print('Unknown vsini_std_err set to 1. km/s.')
    vsinixstd=np.nanstd(vsinix) # !!!!! standard
    if vsinixw==0.0: #if no width-to-vsini plots worked, then probably vsini_target < vsini_std, and this result should be thrown out.
        vsinixw2=float('nan')
        vsinixstd2=float('nan')
        vsiniflag='(NULL AND VOID, vsini_std > vsini_target)'
    elif vsinixstd<0.1: #arbitrary
        vsinixw2=vsinixw
        vsinixstd2=0.1
        vsiniflag='(vsinierr adjusted to 0.1 km/s)'
    else:
        vsinixw2=vsinixw
        vsinixstd2=vsinixstd
        vsiniflag=''
    vsinilen=len([a for a in vsinix if np.isnan(a)==False])
    print('\nWeighted rv of',name,' (',stdspt,') :',rvxw,'+-',rvxstd,'km/s')
    print('Weighted vsini of',name,' (',stdspt,') :',vsinixw,'+-',vsinixstd,'km/s',vsiniflag,'\n')

    return stdname,stdspt,rvxw,rvxstd,vsinixw2,vsinixstd2,bados,vsinilen,skipedge0

In [5]:
# One target against multiple standards:
mname,mnote=opendat(cdir,'standards/standards_metadata.dat',('#name','notes'))
def fitvsini(name,spt,rvcent0=0,vsinistart0=0,skipedge0=0,prin='n',plotyn='n'): #target name, spectral type (including lum. class). Need standards all loaded up.
    print(name,spt)
    print('\nMeasuring rv and vsini.')
    svalue=sval(spt)
    #take all standards within 2 sval of svalue. Also, don't use high-vsini or extreme rad vel standards.
    standards=[stds[i] for i in range(len(stds)) if abs(ssval[i]-svalue)<=2 and 'HIGHVSINI' not in mnote[mname.index(stds[i])] and 'EXTREMERADVEL' not in mnote[mname.index(stds[i])]]
    num=len(standards)
    if num==0:
        print('No suitable standards found. No measurements can be made.')
    else:
        print('Standards:',standards,'\n')
    stdnames,stdspts,rvs,rverrs,vsinis,vsinierrs,bados,vsinilen,skipage=['',]*num,['',]*num,['',]*num,['',]*num,['',]*num,['',]*num,['',]*num,['',]*num,['',]*num
    skipedgestd=0
    for i in range(num):
        done='n'
        print(name,standards[i])
        while done=='n':
            try:
                stdnames[i],stdspts[i],rvs[i],rverrs[i],vsinis[i],vsinierrs[i],bados[i],vsinilen[i],skipage[i]=fitvsinilite(name,standards[i],rvcent0,vsinistart0,skipedgestd,prin=prin,plotyn=plotyn,pltcc=plotyn) #'target','standard'
                done='y'
            except PE.PyAValError:
                print('Fitvsini failed, increasing skipedge...')
                skipedgestd+=5
    flags=[stdnames[i] if abs(rvs[i]-np.median(rvs))>np.std(rvs) and abs(rvs[i]-np.median(rvs))>0.3 else '0' for i in range(len(rvs))]
    rv=erm(rvs,rverrs)[0]
    rverr=np.max([np.std(rvs),erm(rvs,rverrs)[1]]) #spread of rvs is preferred uncert, while erm uncert is lower limit
    vsini=erm(vsinis,vsinierrs)[0]
    vsinierr=np.max([np.std(vsinis),erm(vsinis,vsinierrs)[1]]) # "" for vsinis
    
    #favor vsini measurements that utilize more orders:
    #combined by # of orders utilized:
    #vsini=np.nansum(np.array(vsinis)*np.array(vsinilen)/np.nansum(vsinilen))
    #vsinierr=np.sqrt(np.nansum((np.array(vsinierrs)*np.array(vsinilen)/np.nansum(vsinilen)**2.)))
    
    if num>0:
        bestspt=stdspts[np.argmin(rverrs)]
    else:
        bestspt='nan' #for no similar good standards
    return name,rv,rverr,vsini,vsinierr,bestspt,stdnames,stdspts,rvs,rverrs,vsinis,vsinierrs,bados,vsinilen,flags

['#name', 'spt', 'V', 'texp', 'ra', 'dec', 'vsini', 'vsinierr', 'rv', 'rverr', 'vsiniref', 'rvref', 'sptref', 'notes']


# Measure Radial Velocities

In [6]:
#Measure all targets, and standards too
#Metadata for targets:
targnames,targspts=opendat('','file.dat',['#name','spt'])

In [ ]:
targs=fnamest

#blank lists, for collecting results:
whichnames,whichstds,stdrvs,stdrverrs,stdvsinis,stdvsinierrs,stdbestspts,stdflags,vsinilens=[],[],[],[],[],[],[],[],[]

t0=time.time() 
for i in range(len(targs)):
    if i%10==0:
        t1 = time.time()
        print('Working on '+str(i+1)+' of '+str(len(stds))+', time elapsed:',t1-t0,'s')
    Name=targs[i]
    if 'CG' not in Name:
        SpT=refspt(targs)
    else:
        SpT=targspts[targnames.index(Name)]
    #if sval(stdSpT)<=46:    #45: for SpT A5 and later
    #print(stdName,stdSpT)
    print('\n---\n')
    whichname,rv,rverr,vsini,vsinierr,bestspt,stdnames,stdspts,rvs,rverrs,vsinis,vsinierrs,bados,vsinilen,flags=fitvsini(Name,SpT)
    whichnames.append(whichname)
    whichstds.append(stdnames)
    stdrvs.append(rvs)
    stdrverrs.append(rverrs)
    stdvsinis.append(vsinis)
    stdvsinierrs.append(vsinierrs)
    stdbestspts.append(bestspt)
    stdflags.append(flags)
    vsinilens.append(vsinilen)
    print()
t1 = time.time()
print('Time elapsed:',(t1-t0)/3600.,'hr.') #Took ~5 hr for 99 spectral standards.
#Trial 1: Time elapsed: 12000.443932294846 s = 3h 20m
#Trial 2: 3h 7m